# _Scala Fundamentals_
# 3. Collections and the Option type

_This module was originally developed by Martin Snyder._

## Example Context

Let's start out with this context. We haven't talked about classes or objects yet.
We don't need an in-depth understanding of them for this example, but a brief
description is worthwhile.

- A `case class` is a container for data, with some methods automatically
  generated. (You can add your own, too.)
    - If you're a Java programmer, think of a case class as a readonly Java Bean.
    - If you're a Python programmer, they're kind of like named tuples or the
      [dataclasses](https://docs.python.org/3/library/dataclasses.html) from Python 3.7.
- Case classes are immutable, by default, and every member is public, by default.
- A case class can be instantiated without using the `new` keyword.
- An `object` is a singleton that can contain classes, values, other objects, and
  functions. (The JVM doesn't allow functions to stand alone, by themselves. They have
  to be contained within _something_.)



In [2]:
object Example {
  case class Person(firstName: String, lastName: String, salary: Int, ssn: String) {
    override def toString: String = s"$firstName $lastName"
  }

  val AllPeople = List(
    Person("Abby",   "Adams",    203190, "913-98-6520"),
    Person("Brian",  "Bosworth",  57945, "955-66-1239"),
    Person("Clair",  "Cassidy",  107455, "942-90-6988"),
    Person("Dan",    "Dirkes",   207472, "978-38-9209"),
    Person("Elaine", "Edwards",  231743, "943-68-7575")
  )
}

defined object Example

## Example 3.1: Overview

Problem Statement: Produce a map of SSN &rarr; `Person` for all the "high earners". A high earner is someone earning more than $200,000 a year.

### Solution 3.1a

A naive solution, using mutable collections.

In [3]:
import scala.collection.mutable.HashMap

def buildMapOfHighEarners(people: List[Example.Person]): HashMap[String, Example.Person] = {
  val highEarners = new HashMap[String, Example.Person]()

  people.foreach(person => {
    if (person.salary > 200000) {
      highEarners.put(person.ssn, person)
    }
  })

  highEarners
}

val highEarners = buildMapOfHighEarners(Example.AllPeople)

import scala.collection.mutable.HashMap


defined function buildMapOfHighEarners
highEarners: HashMap[String, Example.Person] = Map(
  "913-98-6520" -> Person("Abby", "Adams", 203190, "913-98-6520"),
  "978-38-9209" -> Person("Dan", "Dirkes", 207472, "978-38-9209"),
  "943-68-7575" -> Person("Elaine", "Edwards", 231743, "943-68-7575")
)

### Solution 3.1b

Functional solution using immutable collections

In [4]:
val highEarners = Example.AllPeople
  .filter(person => person.salary > 200000)
  .map(person => person.ssn -> person)
  .toMap

highEarners: Map[String, Example.Person] = Map(
  "913-98-6520" -> Person("Abby", "Adams", 203190, "913-98-6520"),
  "978-38-9209" -> Person("Dan", "Dirkes", 207472, "978-38-9209"),
  "943-68-7575" -> Person("Elaine", "Edwards", 231743, "943-68-7575")
)

### Solution 3.1c

Alternate example, using a `groupBy` method defined on collections.

In [8]:
val highEarners = Example.AllPeople
  .filter(person => person.salary > 200000)
  .groupBy(person => person.ssn)
  .map { case (ssn, list) => (ssn, list.head) }

highEarners: Map[String, Example.Person] = Map(
  "913-98-6520" -> Person("Abby", "Adams", 203190, "913-98-6520"),
  "978-38-9209" -> Person("Dan", "Dirkes", 207472, "978-38-9209"),
  "943-68-7575" -> Person("Elaine", "Edwards", 231743, "943-68-7575")
)

Don't worry about that `case (ssn, list)` thing. We'll discuss that later.

Example 1c just illustrates that there are often many ways to do the same thing with collections.

## Collections Library

### Common classes

- [`scala.collection.immutable.List`](https://www.scala-lang.org/files/archive/api/2.13.0/scala/collection/immutable/List.html)
- [`scala.collection.immutable.Map`](https://www.scala-lang.org/files/archive/api/2.13.0/scala/collection/immutable/Map.html)
- [`scala.collection.immutable.Set`](https://www.scala-lang.org/files/archive/api/2.13.0/scala/collection/immutable/Set.html)

Aliases for these three definitions also exist in a package that Scala imports for you automatically, so you don't
have to import anything to use them.

### Advantages of immutable collections

- Zero-cost copy operations.
- Free "snapshots" of data.
- Clearer picture of data transitions.
- Related structures can share elements.

## Example 3.2: Adding to a collection

Add a new person to `AllPeople`.

Really, what we'll be doing is creating a new (shallow) copy of `AllPeople`, with the new person added.

### Solution 3.2a

Mutable collection solution.

In [9]:
import scala.collection.mutable.ListBuffer

def addPerson(existingPeople: List[Example.Person], newPerson: Example.Person): List[Example.Person] = {
  val updatedPeople = new ListBuffer[Example.Person]()

  existingPeople.foreach(person => updatedPeople.append(person))
  updatedPeople.append(newPerson)

  updatedPeople.toList
}

val evenMorePeople = addPerson(Example.AllPeople, Example.Person("William", "Gates", 0, "123-45-6789"))

import scala.collection.mutable.ListBuffer


defined function addPerson
evenMorePeople: List[Example.Person] = List(
  Person("Abby", "Adams", 203190, "913-98-6520"),
  Person("Brian", "Bosworth", 57945, "955-66-1239"),
  Person("Clair", "Cassidy", 107455, "942-90-6988"),
  Person("Dan", "Dirkes", 207472, "978-38-9209"),
  Person("Elaine", "Edwards", 231743, "943-68-7575"),
  Person("William", "Gates", 0, "123-45-6789")
)

### Solution 3.2b

Immutable list solution, prepending the new element.

In [10]:
val evenMorePeople = Example.Person("William", "Gates", 0, "123-45-6789") :: Example.AllPeople

evenMorePeople: List[Example.Person] = List(
  Person("William", "Gates", 0, "123-45-6789"),
  Person("Abby", "Adams", 203190, "913-98-6520"),
  Person("Brian", "Bosworth", 57945, "955-66-1239"),
  Person("Clair", "Cassidy", 107455, "942-90-6988"),
  Person("Dan", "Dirkes", 207472, "978-38-9209"),
  Person("Elaine", "Edwards", 231743, "943-68-7575")
)

### Solution 3.2c

Adding to the end of a list.

In [11]:
val evenMorePeople = Example.AllPeople ::: List(Example.Person("William", "Gates", 0, "123-45-6789"))

evenMorePeople: List[Example.Person] = List(
  Person("Abby", "Adams", 203190, "913-98-6520"),
  Person("Brian", "Bosworth", 57945, "955-66-1239"),
  Person("Clair", "Cassidy", 107455, "942-90-6988"),
  Person("Dan", "Dirkes", 207472, "978-38-9209"),
  Person("Elaine", "Edwards", 231743, "943-68-7575"),
  Person("William", "Gates", 0, "123-45-6789")
)

Generally, with a `List`, prepending performs better.

## Example 3.3: Building Sets and Maps

In [12]:
val billGates = Example.Person("William", "Gates", 0, "123-45-6789")
val tuple: (String, Example.Person) = billGates.ssn -> billGates

val setOfPeople1: Set[Example.Person] = Example.AllPeople.toSet
val setOfPeople2: Set[Example.Person] = Set(billGates)

val mapOfPeople1: Map[String, Example.Person] = Map(tuple)
val mapOfPeople2: Map[String, Example.Person] = List(tuple).toMap

billGates: Example.Person = Person("William", "Gates", 0, "123-45-6789")
tuple: (String, Example.Person) = (
  "123-45-6789",
  Person("William", "Gates", 0, "123-45-6789")
)
setOfPeople1: Set[Example.Person] = Set(
  Person("Abby", "Adams", 203190, "913-98-6520"),
  Person("Brian", "Bosworth", 57945, "955-66-1239"),
  Person("Elaine", "Edwards", 231743, "943-68-7575"),
  Person("Clair", "Cassidy", 107455, "942-90-6988"),
  Person("Dan", "Dirkes", 207472, "978-38-9209")
)
setOfPeople2: Set[Example.Person] = Set(
  Person("William", "Gates", 0, "123-45-6789")
)
mapOfPeople1: Map[String, Example.Person] = Map(
  "123-45-6789" -> Person("William", "Gates", 0, "123-45-6789")
)
mapOfPeople2: Map[String, Example.Person] = Map(
  "123-45-6789" -> Person("William", "Gates", 0, "123-45-6789")
)

## Commonly used collection methods

- `filter`
- `map`
- `flatten`
- `flatMap`
- `size` (and, in some cases, `length`)

See [`scala.collection.Iterable`](https://www.scala-lang.org/files/archive/api/2.13.0/scala/collection/Iterable.html).

## Example 3.4: Filter

Create a collection of the people whose first name is one letter shorter than their last name.

(It's higher-order function time, again!)

In [13]:
val somePeople = Example.AllPeople.filter(person => person.firstName.length == person.lastName.length - 1)

somePeople: List[Example.Person] = List(
  Person("Abby", "Adams", 203190, "913-98-6520"),
  Person("Elaine", "Edwards", 231743, "943-68-7575")
)

## Example 3.5: Map

Transform a list of `Person` objects to a collection of SSN (`String`) values.

Remember our `transform` function, from the previous lesson? The `map` function is why
we don't need `transform`. Every collection (and some other things, like `String`) have `map`.

In [14]:
val ssns: Seq[String] = Example.AllPeople.map(person => person.ssn)

ssns: Seq[String] = List(
  "913-98-6520",
  "955-66-1239",
  "942-90-6988",
  "978-38-9209",
  "943-68-7575"
)

## Example 3.6: Flatten

Transform a list of integers into a list of the factors of those integers. Let's do this in steps, so you can see what's happening.

**Note**: The `to` and `until` functions, defined on integral types (and characters) create _ranges_. We're using them here.
Ranges contain their endpoints and a step (which defaults to 1). They don't contain every number in the range. The numbers in
the range are generated lazily, as you loop over the range.

- `1 to 10` creates a range of numbers from 1 to 10, inclusive.
- `1 until 10` creates a range of numbers from 1 to 9.

In [15]:
def getFactors(number: Int): Seq[Int] = {
  (2 until number)
    .filter(candidate => number % candidate == 0)
}

val numbers = (11 to 20 by 3)
val factors = numbers.map(getFactors)
val flattenedFactors = factors.flatten

defined function getFactors
numbers: Range = Vector(11, 14, 17, 20)
factors: collection.immutable.IndexedSeq[Seq[Int]] = Vector(
  Vector(),
  Vector(2, 7),
  Vector(),
  Vector(2, 4, 5, 10)
)
flattenedFactors: collection.immutable.IndexedSeq[Int] = Vector(
  2,
  7,
  2,
  4,
  5,
  10
)

- Notice how each call to `getFactors` for one number returns a `List` of factors for that number.
- Our `numbers.map` call maps a list of numbers into a list of lists of factors.
- Calling flatten on the list of lists does what you'd expect: flattens one dimension out.

(Calling `flatten` on a list of list of lists would produce a list of lists. Each call flattens one level.)

Let's put the whole thing together into one chain. While we're at it, let's filter out duplicates and sort
the result.

In [16]:
val distinctSortedFactors =
  (11 to 20 by 3)
    .map(getFactors)
    .flatten
    .distinct
    .sorted

distinctSortedFactors: collection.immutable.IndexedSeq[Int] = Vector(
  2,
  4,
  5,
  7,
  10
)

## Example 3.7: FlatMap

Transform a list of integers into a list of the factors _of the factors_ of those integers.

In [17]:
def getFactors(number: Int): Seq[Int] = {
  (2 until number)
    .filter(candidate => number % candidate == 0)
}

val distinctSortedFactorsOfFactors =
  (11 to 20 by 3)
    .flatMap(getFactors)
    .flatMap(getFactors)
    .distinct
    .sorted

defined function getFactors
distinctSortedFactorsOfFactors: collection.immutable.IndexedSeq[Int] = Vector(
  2,
  5
)

## Option

`Option` is somewhat like Java's [`Optional`](https://docs.oracle.com/javase/8/docs/api/java/util/Optional.html). Think
of it as either:

- A container for a value that may or may not be present (a kind of type-safe `null`)
- A collection of _at most_ one element.

`Option` is a generic type. You can have:

- an `Option[Int]`: this value may or may not contain an integer
- an `Option[String]`: this value may or may not contain an integer
- an `Option[Map[String, String]]`: this value may or may not contain a `Map`
- etc.

A value of type `Option[T]` can have two possible states:

- `None` (formally, `None[T]`, but we usually leave the type annotation of the `None`): There's no value inside the `Option`.
- `Some[T]`: There's a value inside the `Option`.

Because you can treat an `Option` as a collection, you can call `map`, `filter`, `flatMap`, etc., on it

## Example 3.8: Handling unexpected input

Let's write a function that operates safely on a potentially missing `Person`.

### Solution 3.8a

Traditional approach, using `null`.

In [18]:
def getFirstName(person: Example.Person): String = {
  if (person != null) {
    person.firstName
  }
  else {
    null
  }
}

val shouldBeNull = getFirstName(null)
val shouldBeBill = getFirstName(Example.Person("William", "Gates", 0, "123-45-6789"))

defined function getFirstName
shouldBeNull: String = null
shouldBeBill: String = "William"

The problem with this solution is that we risk `NullPointerException`.

### Solution 3.8b

Naive approach using `Option`: Test whether we have a `Some` or a `None` using `Option.isDefined`. If we have
a `Some`, call `get` to extract the value.


In [ ]:
def getFirstName(person: Option[Example.Person]): String = {
  if (person.isDefined) {
    return person.get.firstName
  }
  else {
    return ""
  }
}

val shouldBeBlank = getFirstName(None)
val shouldBeBill = getFirstName(Some(Example.Person("William", "Gates", 0, "123-45-6789")))

### Solution 3.8c

Functional approach using `Option`:

- Call `map` on the `Option`.
- If we have a `None`, the `map` won't do anything (because the "collection" is empty). The type will change, though.
- If we have a `Some`, we can use the `map` to extract the value.

In [19]:
def getFirstName(person: Option[Example.Person]): Option[String] =
  person.map(p => p.firstName)

val shouldBeNone = getFirstName(None)
val shouldBeBill = getFirstName(Some(Example.Person("William", "Gates", 0, "123-45-6789")))

defined function getFirstName
shouldBeNone: Option[String] = None
shouldBeBill: Option[String] = Some("William")

## Exercise 3.9: Handling unsafe values from unsafe sources

Use `Option` to represent handle sentinel values (including `null`).

### Solution 3.9a

Naive approach using conditionals.

In [20]:
def cleanup(unsafeString: String): Option[String] = {
  if (unsafeString == null || unsafeString == "")
    None
  else
    Some(unsafeString) // now safe!
}

val ex1 = cleanup(null)
val ex2 = cleanup("")
val ex3 = cleanup("Actual Value")

defined function cleanup
ex1: Option[String] = None
ex2: Option[String] = None
ex3: Option[String] = Some("Actual Value")

### Solution 3.9b

Functional approach using factory method and `filter`. We're relying
on the fact that `Option(null)` returns a `None`.

In [22]:
Option(null)

res21: Option[Null] = None

In [21]:
def cleanup(unsafeString: String): Option[String] = {
  Option(unsafeString).filter(s => !s.isEmpty)
}

val ex1 = cleanup(null)
val ex2 = cleanup("")
val ex3 = cleanup("Actual Value")

defined function cleanup
ex1: Option[String] = None
ex2: Option[String] = None
ex3: Option[String] = Some("Actual Value")

## Retrieving values from collections

- Conditionals with unsafe gets
- Safe gets
- Pattern matching

## Exercise 3.10: Unsafe gets

In [23]:
def addFive(i: Option[Int]): Int = {
  i.get + 5 // What happens if i is None?
}

def safeAddFive(maybeInt: Option[Int]): Int = {
  if (maybeInt.isDefined)
    maybeInt.get + 5
  else
    5
}

val ten = addFive(Some(5))
// Uncomment the below line to see how addFive is unsafe.
// val five = addFive(None)
val safeTen = safeAddFive(Some(5))
val safeFive = safeAddFive(None)

defined function addFive
defined function safeAddFive
ten: Int = 10
safeTen: Int = 10
safeFive: Int = 5

### Getting the first element of a collection

Calling `head` on a collection attempts to retrieve the first element. What happens if the
collection is empty?

In [26]:
val numbers = List(1, 2, 3)
val noNumbers = List.empty[Int]

val one = numbers.head
// val oops = noNumbers.head

numbers: List[Int] = List(1, 2, 3)
noNumbers: List[Int] = List()
one: Int = 1

It's better to use `headOption`

In [27]:
numbers.headOption
noNumbers.headOption

res26_0: Option[Int] = Some(1)
res26_1: Option[Int] = None

### Getting values from a map

- Unsafe: "call" a map to get a value for a key
- Safe: use `get`.

In [28]:
val numbers: Map[String, Int] = Map(
  "one"   -> 1,
  "two"   -> 2,
  "three" -> 3
)

val safe1 = numbers.get("two")
val unsafe1 = numbers("two")
val safe2 = numbers.get("fifteen")
//val unsafe2 = numbers("fifteen")

numbers: Map[String, Int] = Map("one" -> 1, "two" -> 2, "three" -> 3)
safe1: Option[Int] = Some(2)
unsafe1: Int = 2
safe2: Option[Int] = None

## Exercise 11: Safe gets

Use `getOrElse` to supply default values

In [29]:
def addFive(maybeInt: Option[Int]): Int =
  maybeInt.getOrElse(0) + 5

val ten = addFive(Some(5))
val five = addFive(None)

defined function addFive
ten: Int = 10
five: Int = 5

In [31]:
val numbers = List(1, 2, 3)
val noNumbers = List.empty[Int]

val one = numbers.headOption.getOrElse(0)
val zero = noNumbers.headOption.getOrElse(0)

numbers: List[Int] = List(1, 2, 3)
noNumbers: List[Int] = List()
one: Int = 1
zero: Int = 0

In [32]:
val numbers: Map[String, Int] = Map(
  "one"   -> 1,
  "two"   -> 2,
  "three" -> 3
)

val trial: Int = numbers.get("two").getOrElse(2)
val two: Int = numbers.getOrElse("two", 2)

numbers: Map[String, Int] = Map("one" -> 1, "two" -> 2, "three" -> 3)
trial: Int = 2
two: Int = 2

## Recap

## Exercise 3.12

Extra `flatMap` example.

Recall the contents of `Example.AllPeople`.

In [33]:
Example.AllPeople

res32: List[Example.Person] = List(
  Person("Abby", "Adams", 203190, "913-98-6520"),
  Person("Brian", "Bosworth", 57945, "955-66-1239"),
  Person("Clair", "Cassidy", 107455, "942-90-6988"),
  Person("Dan", "Dirkes", 207472, "978-38-9209"),
  Person("Elaine", "Edwards", 231743, "943-68-7575")
)

In [34]:
val Beneficiaries = Map(
  "913-98-6520" -> "955-66-1239",
  "942-90-6988" -> "978-38-9209",
  "943-68-7575" -> "000-00-0000"
)

def getPersonBySSN(ssn: String) = Example.AllPeople.find(person => person.ssn == ssn)

val beneficiaries: List[Example.Person] = Example.AllPeople
  .flatMap(person => Beneficiaries.get(person.ssn))
  .flatMap(ssn => getPersonBySSN(ssn))

Beneficiaries: Map[String, String] = Map(
  "913-98-6520" -> "955-66-1239",
  "942-90-6988" -> "978-38-9209",
  "943-68-7575" -> "000-00-0000"
)
defined function getPersonBySSN
beneficiaries: List[Example.Person] = List(
  Person("Brian", "Bosworth", 57945, "955-66-1239"),
  Person("Dan", "Dirkes", 207472, "978-38-9209")
)